In [33]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp

In [34]:
class OneToOneITU():
    def __init__(self, n_i, m_j, parameters = (None,None) , lbs=(0, 0)):
        self.n_i = n_i
        self.m_j = m_j
        self.lb_U, self.lb_V = lbs

        self.n = len(n_i)
        self.m = len(m_j)
       
        self.A_ij = parameters[0]
        self.B_ij = parameters[1]

    def D_ij(self, U_i, V_j):
        return (U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij)/(1+ self.A_ij)

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, j_idx]

    def get_V_ij(self,i_idx, j_idx , U_i):
        return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]

In [35]:
def OT_auction(self, data = None, tol_ε = None):
    if data is None:
        V_j = np.ones(self.m) * self.lb_V
        mu_ij = np.zeros([self.n_i, self.m_j], dtype= int)

        

In [36]:
np.random.seed(200)
n_i = np.random.randint(1,4, size = 5)
m_j = np.random.randint(1,4, size = 5)
A = np.random.choice([.25,5,1,2,4], size= [len(n_i), len(m_j)] )
B = np.random.randint(1,5, size = [len(n_i), len(m_j)])

In [37]:
example_mkt = OneToOneITU( n_i,m_j, parameters=(A,B))

In [38]:
y_ij = np.zeros([example_mkt.n+1, example_mkt.m])
f_ij = np.zeros([example_mkt.n+1, example_mkt.m+1], dtype= int)
f_ij[-1,:] = np.append(example_mkt.m_j,0)
f_ij[-1,-1] = example_mkt.n+200

In [39]:
# free_capacity_i = example_mkt.n_i -  f_ij[:-1,:].sum(1)
# bidder_i =  np.where(free_capacity_i > 0)[0]

# U_ikj = (example_mkt.B_ij[bidder_i,None,:] - example_mkt.A_ij[bidder_i,None,:] * np.where( f_ij[:,:-1] >0 ,y_ij, np.nan ) )
# U_ikj[np.arange(len(bidder_i)),np.arange(len(bidder_i))] = np.nan

# U_i_kj0 = np.concatenate((U_ikj.reshape(len(bidder_i), (example_mkt.n+1) * example_mkt.m), np.ones((len(bidder_i),1))* example_mkt.lb_U), axis= 1)
# U_i_kj0_sorted = np.argsort(-U_i_kj0, axis= 1 )

In [40]:
# i_l = (U_i_kj0_sorted )// (example_mkt.m)
# j_l = (U_i_kj0_sorted )%(example_mkt.m)
# ell_i = np.zeros( len(bidder_i), dtype= int)
# residual_i = free_capacity_i[bidder_i]
# f_ij_hat = f_ij[bidder_i].copy()

In [41]:
# while np.any(residual_i >=0):

#     f_il_jl = f_ij[i_l[range(len(bidder_i)),ell_i], j_l[range(len(bidder_i)),ell_i]]

#     f_ij_hat[:, -1] += (i_l[range(len(bidder_i)),ell_i] == example_mkt.n+1) * residual_i * (residual_i >=0)

#     f_ij_hat[range(len(bidder_i)), j_l[range(len(bidder_i)),ell_i]] += (i_l[range(len(bidder_i)),ell_i] != example_mkt.m+1)*( f_il_jl *(residual_i >= f_il_jl) + 
#                                         residual_i * (residual_i < f_il_jl) * ( residual_i >= 0))

#     ell_i += (i_l[range(len(bidder_i)),ell_i] != example_mkt.m+1)*(residual_i >= 0) 
#     residual_i -= f_il_jl + (residual_i - f_il_jl)*((i_l[range(len(bidder_i)),ell_i] == example_mkt.n+1)) 

In [42]:
# j_k_case2 = np.argmax(j_l != np.argmax(f_ij_hat , axis= 1)[:,None], axis= 1)

# w_i = ((U_i_kj0[range(len(bidder_i)),U_i_kj0_sorted[range(len(bidder_i)),ell_i] ] * ((f_ij_hat[:,:-1]>0).sum(1) >1) +
#         U_i_kj0[range(len(bidder_i)),U_i_kj0_sorted[range(len(bidder_i)),j_k_case2]] * ((f_ij_hat[:,:-1]>0).sum(1) == 1) * (i_l[range(len(bidder_i)),j_k_case2] != example_mkt.m +1)
#         + example_mkt.lb_U * ((f_ij_hat[:,:-1]>0).sum(1) == 1) * (i_l[range(len(bidder_i)),j_k_case2] == example_mkt.m +1)
#        )*(i_l[range(len(bidder_i)),ell_i] != example_mkt.m+1)
#        + example_mkt.lb_U * (i_l[range(len(bidder_i)),ell_i] == example_mkt.m+1)
#        )

In [43]:
# bids = np.where(f_ij_hat[:,:-1]> 0, (example_mkt.B_ij - w_i[:,None])/example_mkt.A_ij, np.nan)

In [44]:
def bidding_phase(self, f_ij,y_ij, tol_ε):
    free_capacity_i =self.n_i -  f_ij[:-1,:].sum(1)
    bidder_i =  np.where(free_capacity_i > 0)[0]

    U_ikj = (self.B_ij[bidder_i,None,:] - self.A_ij[bidder_i,None,:] * np.where( f_ij[:,:-1] >0 ,y_ij, np.nan ) )
    U_ikj[np.arange(len(bidder_i)),np.arange(len(bidder_i))] = np.nan
    #print(np.shape(U_ikj))
    #U_i_kj0 = np.concatenate((U_ikj.reshape(self.n, (self.n+1) * self.m), np.ones((self.n,1))* self.lb_U), axis= 1)
    U_i_kj0 = np.concatenate((U_ikj.reshape(len(bidder_i), (self.n+1) * self.m), np.ones((len(bidder_i),1))* self.lb_U), axis= 1)
    U_i_kj0_sorted = np.argsort(-U_i_kj0, axis= 1 )


    i_l = (U_i_kj0_sorted )// (self.m)
    j_l = (U_i_kj0_sorted )%(self.m)
    ell_i = np.zeros( len(bidder_i), dtype= int)
    residual_i = free_capacity_i[bidder_i]
    f_ij_hat = f_ij[bidder_i].copy()

    while np.any(residual_i >=0):

        f_il_jl = f_ij[i_l[range(len(bidder_i)),ell_i], j_l[range(len(bidder_i)),ell_i]]
        f_ij_hat[:, -1] += (i_l[range(len(bidder_i)),ell_i] == self.n+1) * residual_i * (residual_i >= 0)
        f_ij_hat[range(len(bidder_i)), j_l[range(len(bidder_i)),ell_i]] += ((i_l[range(len(bidder_i)),ell_i] != self.m+1)*
                                                                        ( f_il_jl *(residual_i >= f_il_jl) + residual_i * (residual_i < f_il_jl))* (residual_i >= 0))
        ell_i += (i_l[range(len(bidder_i)),ell_i] != self.m+1)*(residual_i >= 0) 
        residual_i -= f_il_jl + (residual_i - f_il_jl)*((i_l[range(len(bidder_i)),ell_i] == self.n+1)) 

    j_k_case2 = np.argmax(j_l != np.argmax(f_ij_hat , axis= 1)[:,None], axis= 1)

    w_i = ((U_i_kj0[range(len(bidder_i)),U_i_kj0_sorted[range(len(bidder_i)),ell_i] ] * ((f_ij_hat[:,:-1]>0).sum(1) >1) +
            U_i_kj0[range(len(bidder_i)),U_i_kj0_sorted[range(len(bidder_i)),j_k_case2]] * ((f_ij_hat[:,:-1]>0).sum(1) == 1) * (i_l[range(len(bidder_i)),j_k_case2] != self.m +1)
            + self.lb_U * ((f_ij_hat[:,:-1]>0).sum(1) == 1) * (i_l[range(len(bidder_i)),j_k_case2] == self.m +1)
        )*(i_l[range(len(bidder_i)),ell_i] != self.m+1)
        + self.lb_U * (i_l[range(len(bidder_i)),ell_i] == self.m+1)
        ) - tol_ε
    
    bids = np.where(f_ij_hat[:,:-1]> 0, (self.B_ij[bidder_i,:] - w_i[:,None])/self.A_ij[bidder_i,:], np.nan)

    return bids, f_ij_hat,bidder_i

OneToOneITU.bidding_phase = bidding_phase

In [45]:
# `bids_ij, f_hat_ij , bidder_i  = example_mkt.bidding_phase(f_ij, y_ij,.01)

In [46]:
# f_ij , y_ij = example_mkt.assignment_phase(f_ij, y_ij, bids_ij, f_hat_ij)

In [47]:
# received_bid_j = np.where(np.any(np.isnan(bids_ij ) == False , axis= 0))[0]

# P_ij = np.ones_like(y_ij)*np.nan
# P_ij[:,received_bid_j] = np.where(f_ij[:,received_bid_j]>0, y_ij[:,received_bid_j] , np.nan)
# P_ij[bidder_i,:] = np.where(f_hat_ij[:,:-1]>0, bids_ij, np.nan ) 

In [48]:
# f_hat_ij_2 = np.zeros_like(f_ij)
# f_hat_ij_2[:-1,received_bid_j] = np.where(f_ij[:-1,received_bid_j]>0, f_ij[:-1,received_bid_j] , 0)
# f_hat_ij_2[bidder_i,:] = f_hat_ij.copy()
# P_ij = np.concatenate((P_ij, np.where(f_hat_ij_2[:,-1] >0, 0,np.nan )[:,None]), axis= 1)

In [49]:
# P_ij_sorted_id = np.argsort(- P_ij , axis= 0)
# ell_j = np.zeros(example_mkt.m+1, dtype=int)

In [50]:
# done_j = np.ones(example_mkt.m+1, dtype= bool)
# done_j[received_bid_j] = False

# while np.any(done_j == False):
#     for j in received_bid_j:
        
#         f_ij_hat_ell = f_hat_ij_2[P_ij_sorted_id[ell_j[j],j],j]
#         residual_j = f_ij[-1,j]
#         if f_ij_hat_ell > 0 and residual_j > 0:
#             print("lulz")
#             f_ij[P_ij_sorted_id[ell_j[j],j],j] = f_ij_hat_ell * (residual_j>= f_ij_hat_ell) + residual_j * (residual_j < f_ij_hat_ell) 
#             f_ij[-1,j] -= np.minimum(f_ij_hat_ell, residual_j)
#             y_ij[P_ij_sorted_id[ell_j[j],j],j] = P_ij[P_ij_sorted_id[ell_j[j],j],j]
#             ell_j[j]+=1
#         else:
#             done_j[j] = True

In [51]:
def assignment_phase(self,f_ij_, y_ij_, bids_ij, f_hat_ij):
    received_bid_j = np.where(np.any(np.isnan(bids_ij ) == False , axis= 0))[0]

    P_ij = np.ones_like(y_ij)*np.nan
    P_ij[:,received_bid_j] = np.where(f_ij[:,received_bid_j]>0, y_ij[:,received_bid_j] , np.nan)
    P_ij[bidder_i,:] = np.where(f_hat_ij[:,:-1]>0, bids_ij, np.nan ) 
    f_hat_ij_2 = np.zeros_like(f_ij)
    f_hat_ij_2[:-1,received_bid_j] = np.where(f_ij[:-1,received_bid_j]>0, f_ij[:-1,received_bid_j] , 0)
    f_hat_ij_2[bidder_i,:] = f_hat_ij.copy()
    P_ij = np.concatenate((P_ij, np.where(f_hat_ij_2[:,-1] >0, 0,np.nan )[:,None]), axis= 1)

    P_ij_sorted_id = np.argsort(- P_ij , axis= 0)
    ell_j = np.zeros(self.m+1, dtype=int)  

    done_j = np.ones(example_mkt.m+1, dtype= bool)
    done_j[received_bid_j] = False

    while np.any(done_j == False):
        for j in received_bid_j:
            f_ij_hat_ell = f_hat_ij_2[P_ij_sorted_id[ell_j[j],j],j]
            residual_j = f_ij[-1,j]
            if f_ij_hat_ell > 0 and residual_j > 0:
                print("lulz")
                f_ij[P_ij_sorted_id[ell_j[j],j],j] = f_ij_hat_ell * (residual_j>= f_ij_hat_ell) + residual_j * (residual_j < f_ij_hat_ell) 
                f_ij[-1,j] -= np.minimum(f_ij_hat_ell, residual_j)
                y_ij[P_ij_sorted_id[ell_j[j],j],j] = P_ij[P_ij_sorted_id[ell_j[j],j],j]
                ell_j[j]+=1
            else:
                done_j[j] = True
                
    return f_ij, y_ij

OneToOneITU.assignment_phase = assignment_phase

In [55]:
bids_ij, f_hat_ij , bidder_i  = example_mkt.bidding_phase(f_ij, y_ij,.01)

IndexError: index 6 is out of bounds for axis 0 with size 6

In [56]:
f_ij , y_ij = example_mkt.assignment_phase(f_ij, y_ij, bids_ij, f_hat_ij)

In [54]:
f_ij

array([[  1,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   1,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   2,   0,   0,   1,   0],
       [  0,   0,   3,   2,   0, 205]])